<h1>Quick context :</h1> 

I'm trying to build the model of a small Spacecraft using SysML2.
<br>
My reference for doing that is the Powerpoint presentation of August 21: <u>Intro to Pilot SysML v2 Language</u>. I encountered some issues, so my questions will be guided from there.
<br>
<br>
Here are some references to the SysML 1 model :



<hr> 

![containment](https://imgur.com/drgqZzK.jpg) 

![bdd](https://imgur.com/iKteipO.jpg)

![ibd_mission](https://imgur.com/wsKMYn5.jpg)

<hr>

# Going into SysML2

### Recreating the structure :

In [15]:
package 'Structure'{
    package 'Interfaces'{}
    block 'Mission';
    block 'Ground Station';
    block 'Spacecraft Vehicle';
}

Package Structure (3a301f08)


### Value properties, ports or interface definitions (no apparent issues)

In [13]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            //no references ?
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
    }
      
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
        
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

Package Structure (b0c1ad0)


 <hr>

## Defining activities and actions

This is where the troubles begin.<br>
<span style="color:red">For the sake of clarity, I'll just expose the blocks that are useful, the cells won't be runnable anymore, but the full version is available down below.</span><br>
I listed the activities each block contains and tried to depict and formalize the flow of actions happening in said activities.</br>

In [ ]:
    block Data;
    block Signal; //I did not find any other solutions to have signals flowing from one activity to another
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
        
        
        activity PingSpacecraft(out ping:Signal){
            // 1. send signal on CommunicationLink interface 
        } 
        
        activity ReceiveData(in dataPacket:Data){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
        }
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
       
        activity ConsumePower(in deviceOff:Signal, out batteryLow:Signal){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
        }

        activity RechargeBatteries(out fullCharge:Signal){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1.
        }
        
        activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }
        
        
        activity TransmitData(in batteryLow:Signal, 
                              out deviceOff:Signal, 
                              out transmissionDone:Signal){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
        }
        
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }

  <hr>

<b style="color:red">I encountered many issues when trying to actually implement those activities, and I'll now try to go over them one by one.</b>

<h2>Operations on value properties</h2><br>
Is it possible to make operations on the value properties of a block in an action? <br>
An example of a use-case would be on line 1. of SendDataFrames activity:

In [ ]:
activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }

<hr>

<h2>Forks and action flow</h2>
<ul>
    <li>Are forks a thing in SysML2, or is it simply not specifying a 'then' before calling the following action?</li>
    <li>What happens if I call multiple actions in an activity, without calling one 'then' the other? Will they run simultaneously?</li>
    <li>Is the keyword 'then' the best way to represent an action flow inside an activity?</li>
</ul>

<hr>

<h2>Merge and decision nodes</h2>
The following example will be useful:

![act_recharge](https://imgur.com/fKHl12u.jpg)
I tried defining the merge node following the example slide 31:

In [ ]:
activity RechargeBatteries(){
    first start;
    then[0..1] action mergeNode : Activities::Merge;
    // decision node
        // battery++;
        then[0..1] mergeNode;
    //else ...
    }

<h4>Here are some questions:</h4>
<ul>
    <li>Is the merge node correctly defined in this case?</li>
    <li>What is the multiplicity ( [0..1] ) used for? What does it define?</li>
    <li>Is it possible to use Activities::Decision to define a Decision Node, and if so: How?

<hr>

<h2>Sending signals to other parts/activities</h2>
Using the simplest example :

![send_ping](https://imgur.com/GX0t4TL.jpg)

I want the ping to be sent to the Spacecraft Vehicle via the CommunicationInterface on port groundCommIF.<br>
<ul>
    <li>How would one define this activity? (See right down below)</li>
    <li>Is the target "Spacecraft Vehicle", groundPort, spacePort or CommunicationLink?</li>
    <li>Do I need to define a block Signal, or is there an intergrated thing for this kind of signaling?</li>
</ul>

In [ ]:
send ping:Signal to "Spacecraft Vehicle";

^ This line prompted an error requiring an "of" argument.

In [ ]:
send ping:Signal of null to "Spacecraft Vehicle";

^ This ran without error, but what is the null expected to be? This is not referred to in the slides

 <hr>

<h2>Receiving a signal from another part/activity</h2>
<br>
Using the receiving end from the last example, where would I define the trigger? In the SysML1 model, WaitingforGroundStationPing is a state in the state-machine diagram, but never is there an activity that awaits the trigger of the ping.
<br><br>
Using another example:

![receiving_ping](https://imgur.com/ZGEanDG.jpg)

Following slide 30 (Signaling), I would be tempted to write the following:

In [ ]:
activity ReceiveData(in data:Data flow from "Spacecraft Vehicle"::SendDataFrames::data){
    action merge : Activities::Merge;
    accept trigger(data:Data);
    then action processData(data);
    then[0..1] merge;
}

But I'm really not sure I understand how the (accept) trigger works. 
<ul><li>How does the trigger know where to "sniff" for data? Should I pass as argument the port on which I want the trigger to listen instead?</li>
    <li>If not, some indications would be appreciated</li></ul>
        
But the "Spacecraft Vehicle" block is not accessible from this function which I understand, and that bring me to my next question:


<h2>Arguments flowing between activities</h2>
<ul>
    <li>How does one activity accept "in arguments" from activities nested in another block?</li> 
    <li>Do I need to define the activities with 'action' from the Mission block?</li>
</ul>

From my (basic) understanding, the SysML1 model has both the actions and activities defined inside the GroundStation and Spacecraft blocks. And they’re called once the blocks are "instantiated" (part properties) in the Mission block.<br>
But it seems to me like the Action Allocation slide (s. 32) suggests that I should call the activities myself while instantiating the blocks.

### So here is what I have for now, I'm blocked at the activity definition

In [2]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            //no references ?
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    
    
      block Data;
      block Signal; //I did not find any other solutions to have signals flowing from one activity to another
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
        
        
        activity PingSpacecraft(out ping:Signal){
            // 1. send signal on CommunicationLink interface 
        } 
        
        activity ReceiveData(in dataPacket:Data){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
        }
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
       
        activity ConsumePower(in deviceOff:Signal, out batteryLow:Signal){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
        }

        activity RechargeBatteries(out fullCharge:Signal){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1.
        }
        
        activity SendDataFrames(out dataPacket:Data){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
        }
        
        
        activity TransmitData(in batteryLow:Signal, 
                              out deviceOff:Signal, 
                              out transmissionDone:Signal){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
        }
        
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

Package Structure (e2ea939)


In [3]:
%show Structure::"Ground Station"

Block Ground Station (3fd19188)
  [FeatureMembership] PortUsage CommIF (79670fb8)
    [FeatureTyping] PortDefinition CommunicationInterface (4d1d80de)
    [Subsetting] PortUsage ports (3c5dbdf8)
  [Membership] Activity PingSpacecraft (652fc517)
    [Superclassing] Activity Action (23b8d9f3)
  [Membership] Activity ReceiveData (65769740)
    [ParameterMembership] Parameter data (784db959)
      [FeatureTyping] Block Data (549871b2)
      [Subsetting] Feature objects (1f387978)
    [Superclassing] Activity Action (23b8d9f3)
  [FeatureMembership] Action pingSpacecraft (40807d1b)
    [FeatureTyping] Activity PingSpacecraft (652fc517)
    [ParameterMembership] Parameter (6e961cd4)
      [Subsetting] Feature things (453d496b)
    [Subsetting] Action performedActions (2f5ac102)
  [FeatureMembership] Action receiveData (6c77c9b7)
    [FeatureTyping] Activity ReceiveData (65769740)
    [ParameterMembership] Parameter (66c4d917)
      [Redefinition] Parameter data (784db959)
    [FeatureMembersh

In [18]:
%show Structure::"Spacecraft Vehicle"

Block Spacecraft Vehicle (13021680)
  [FeatureMembership] PortUsage CommIF1 (705b2998)
    [FeatureTyping] PortDefinition CommunicationInterface (70750bfa)
    [Subsetting] PortUsage ports (7a0e7ecd)
  [Membership] Activity ConsumePower (ebd959c)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity RechargeBatteries (479e4d69)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity SendDataFrames (6bdac984)
    [ParameterMembership] Parameter data (62f446e5)
      [FeatureTyping] Block Data (64054b3c)
      [Subsetting] Feature objects (2116b68b)
    [Superclassing] Activity Action (1e58512c)
  [Membership] Activity TransmitData (697596ec)
    [Superclassing] Activity Action (1e58512c)
  [FeatureMembership] Action transmitData (5098a17d)
    [FeatureTyping] Activity TransmitData (697596ec)
    [ParameterMembership] Parameter data (12bd2230)
      [FeatureTyping] Block Data (64054b3c)
      [Subsetting] Feature objects (2116b68b)
    [Subsetting] Action 